<a href="https://colab.research.google.com/github/TJTomas/DSPS_TTomaszewski/blob/main/HW9/Homework_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

import ipywidgets as ipw
import numpy as np
import pylab as pl
import sklearn.cluster
from sklearn import mixture
from scipy.cluster.vq import kmeans2
%pylab inline


import json
import os
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# Task 1:
Task 1: make a kaggle account and set up your API (see below)

In [ ]:
# this mounts your google drive
from google.colab import drive
drive.mount("/content/gdrive")



In [ ]:
# this gets you to your drive folder
%cd gdrive/My\ Drive/

In [ ]:
# this makes sure the file is there: this cell should return "kaggle.json"
!ls kaggle.json

In [ ]:
# this limits who can view and make changes who can access this file.
!chmod 600 kaggle.json

# this reads in the file and stores it into the system variables of your colab sessions which allows you to connect programmatically to the kaggle platform
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = "e60b57c215e877e01a22375a3058eec1"#envs['key']

#Task 2
Task 2: read in the data for the World Happniess Dataset https://www.kaggle.com/datasets/unsdsn/world-happiness/data - use the 2024 data

In [ ]:
import kaggle

In [ ]:
# use the kaggle.api.dataset_download_files function to download a specific file from a Kaggle dataset in Jupyter Notebook:
#kaggle.api.dataset_download_files("unsdsn/world-happiness", path="./", unzip=True, quiet=False)
kaggle.api.dataset_download_files("mathurinache/world-happiness-report", path="./", unzip=True, quiet=False)

In [ ]:
happiness = pd.read_csv("2021.csv")

# displaying the 6 raws of contents of the cvs file
happiness.head()

In [ ]:
happiness.shape

In [ ]:
happiness.describe()

# Scaling
## Task 3:
Task 3: For each numerical column X, prepare a column that is a minmax version of X and a version that is the standardized version of X, store them in the same or another dataframe (your choice) as, for example X_minmax and X_standardized (choose the variable or dataframe names you want, but make them meaningful and descriptive!)

In [ ]:
# Define min-max scaling function
def min_max_scale(data):
    min_val = data.min()
    max_val = data.max()
    return (data - min_val) / (max_val - min_val)

# Define standardization function
def standardize(data):
    mean_val = data.mean()
    std_val = data.std()
    return (data - mean_val) / std_val

In [ ]:
numerical_cols = happiness.select_dtypes(include=['float64', 'int64']).columns

# Two new DataFrames to store the scaled columns
scaled_happiness_min_max = pd.DataFrame()
scaled_happiness_std = pd.DataFrame()

# Iterate through numerical columns and apply scaling
for col in numerical_cols:

    # Apply Min-Max scaling
    scaled_happiness_min_max[f'{col}_minmax'] = min_max_scale(happiness[col])

    # Apply Standardization
    scaled_happiness_std[f'{col}_standardized'] = standardize(happiness[col])

# Drop the 'Ladder score in Dystopia_minmax' column as it contains NaN / all zeros
scaled_happiness_min_max = scaled_happiness_min_max.drop(columns=['Ladder score in Dystopia_minmax'])
scaled_happiness_std = scaled_happiness_std.drop(columns=['Ladder score in Dystopia_standardized'])
happiness = happiness.drop(columns=['Ladder score in Dystopia'])

In [ ]:
scaled_happiness_std.head()

In [ ]:
scaled_happiness_min_max.head()

## Task 4:

For each numerical column pair X and Y make a scatter plot of Y vs X with the data as is read in, a scatter plot of Y_minmax vs X_minmax, and a scatter plot of Y_standardized and X_standardized

In [ ]:
def plot_scatter_pairs(dataframe, title, col_args):

    #pd.plotting.scatter_matrix(dataframe, figsize=(15, 15))
    #plt.suptitle('Scatter Matrix Plot')
    #plt.show()


    # Create the scatter plot matrix
    sm = pd.plotting.scatter_matrix(dataframe, figsize=(15, 15))

    if col_args is not None:
      sm = pd.plotting.scatter_matrix(dataframe[col_args], figsize=(15, 15))

    # Rotate labels by 45 degrees
    for ax in sm.ravel():
        ax.xaxis.label.set_rotation(45)
        ax.yaxis.label.set_rotation(45)

    # Adjust y-axis label position
    for i, ax in enumerate(sm[:, 0]): # Only for the first column of subplots (y-axis labels)
        ax.yaxis.set_label_coords(-0.8, 0.5)


    # Adjust x-axis label position for the bottom row
    for i, ax in enumerate(sm[-1, :]): # Only for the bottom row of subplots (x-axis labels)
        ax.xaxis.set_label_coords(0.5, -0.2)


    plt.suptitle(title, y=0.92, fontsize=16)

    plt.show()

In [ ]:
plot_scatter_pairs(happiness, 'World Happiness Report (2021) Scatter Matrix', None)

# Fig 1:
In this figure we are ploting each column data from the world happiness dataset from kaggle in 2021. I was unable to find the dataset from 2024. Here we plot every column against every other column to create our large scatter plot. Each numerical column is plotted against every other column. Each plot is represented by either a scatter plot, or is represented by a histogram in the cases of the data being plotted against itself. We are are abe to see some exact linear relations between some metrics. I am unsure if this is due to some exact scaling law or is due to some kind of data artifacting. However, there seems to be real positive trends in GDP per capita and other metrics seen in the data.

In [ ]:
plot_scatter_pairs(scaled_happiness_std, "World Happiness Report (2021) Scatter Matrix with Standardized Values", None)

# Figure 2:

Again, we are ploting each column data from the world happiness dataset from kaggle in 2021. I was unable to find the dataset from 2024. Here we plot every column against every other column to create our large scatter plot. This time we have standardized our data. We are able to see that our data distributions are conserved from the hapiness set, but now range from -2.5 to 2.5.

In [ ]:

plot_scatter_pairs(scaled_happiness_min_max, "World Happiness Report (2021) Scatter Matrix with Min Max Values", None)

# Figure 3:
Finally, we are ploting each column data from the world happiness dataset from kaggle in 2021. I was unable to find the dataset from 2024. Here we plot every column against every other column to create our large scatter plot. This time we have min maxed our data. The data now ranges from zero to 1 for all of our data, with our distributions kept in tact.

# Clutering
## Task 5:
Task 5: Using KMeans clustering, cluster the scaled numerical features (choose either scaling) that are used to calculate the score: 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual' into 3 clusters

In [ ]:
def cluster_happiness_by_columns(original_df, scaled_df, n_clusters, column_indices):
    # Get the names of the columns to use for clustering based on numerical indices
    scaled_columns = scaled_df.columns
    selected_columns = [scaled_columns[i] for i in column_indices if i < len(scaled_columns)]

    # Print the names of the selected scaled columns
    print("Scaled columns used for clustering:", selected_columns)

    # Select the scaled features for clustering
    scaled_features = scaled_df[selected_columns]

    # Initialize and fit KMeans
    kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    original_df['Cluster'] = kmeans.fit_predict(scaled_features)

    return original_df

In [ ]:
original_df = happiness
scaled_df = scaled_happiness_min_max
n_clusters = 3
column_indices = [5, 6, 7, 9, 8, 17]
cluster_happiness_by_columns(original_df, scaled_df, n_clusters, column_indices)

# Task 6:
Task 6: Make a scatter plot with the cluster (0, 1, or 2) on the X axis, and the Happiness score with its errorbar on the Y axis and, as usual, comment on the figure your what, how, wow

In [ ]:

# Create the scatter plot of individual happiness scores by cluster
plt.figure(figsize=(10, 6))

# Define colors for each cluster
colors = {0: 'purple', 1: 'green', 2: 'orange'} # Use a dictionary for mapping cluster to color

# Plot each cluster separately
for cluster_label in sorted(happiness['Cluster'].unique()):
    cluster_data = happiness[happiness['Cluster'] == cluster_label]
    plt.scatter(
        x=cluster_data['Cluster'],
        y=cluster_data['Ladder score'],
        color=colors[cluster_label],
        label=f'Cluster {cluster_label}',
        s=10
    )
    plt.errorbar(
        x=cluster_data['Cluster'],
        y=cluster_data['Ladder score'],
        yerr=cluster_data['Standard error of ladder score'],
        fmt='none',
        color=colors[cluster_label],
        label=f'Cluster {cluster_label} Score Error'
    )

# Set labels and title
plt.xlabel('Cluster Number')
plt.ylabel('Ladder Score')
plt.title('Individual Happiness Scores by Cluster (2021)')
plt.xticks(sorted(happiness['Cluster'].unique())) # Only show values 0,1,2 on the X axis
plt.legend(title='Cluster')

plt.show()

# Figure 4:
Here, we are plotting our hapiness data that we used KMeans clustering, cluster the scaled numerical features (choose either scaling) that are used to calculate the score: 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual' into 3 clusters, then plotted our three clusters agaisnt our happiness ladder score. Coorisponding error is also included in the plot. The legend outlines as such, with colors coorisponding to each cluster. We are able to see that cluster zero is the elaest happy on average, cluster 1 is happier, and cluster 2 is the most happy on average. Cluster 1 includes the largest range of values, when compared to cluster 0 and 1. There is quite a bit of sharex calues between clusters 1 and the other clusters.